In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
file_path = "/fs/ess/PAS2136/Rayeed/BritishCarabids/British Carabids/BritishCarabids.csv"
brit_df = pd.read_csv(file_path)
brit_df.rename(columns={"CanonicalName": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = brit_df[cols]
df = df.dropna()
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum
1,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Asaphidion curtum


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [5]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of species : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [6]:
X = np.vstack([extract_features(img, taxa) for img, taxa in tqdm(zip(df["ImageFilePath"], df["ScientificName"]), desc="Processing ....") ])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)




Processing ....: 0it [00:00, ?it/s]

In [7]:

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 73.34% | Prec: 78.68% | Rec: 73.34% | F1: 74.52% | Bal Acc: 75.53% | MCC: 0.7323
LogisticRegression        | Acc: 99.22% | Prec: 99.25% | Rec: 99.22% | F1: 99.22% | Bal Acc: 98.84% | MCC: 0.9922


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NearestNeighbor           | Acc: 84.12% | Prec: 85.70% | Rec: 84.12% | F1: 84.10% | Bal Acc: 79.75% | MCC: 0.8404
MLP_Baseline              | Acc: 99.50% | Prec: 99.51% | Rec: 99.50% | F1: 99.50% | Bal Acc: 99.36% | MCC: 0.9950


In [8]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
6930,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Harpalus rubripes,Harpalus rubripes,Harpalus rubripes,Harpalus rubripes,Harpalus rubripes
2549,/fs/ess/PAS2136/Rayeed/BritishCarabids/British...,Bembidion fumigatum,Bembidion fumigatum,Bembidion fumigatum,Bembidion fumigatum,Bembidion fumigatum


In [9]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.733354,0.786783,0.733354,0.745246,0.755334,0.732283
LogisticRegression,LogisticRegression,0.992232,0.992458,0.992232,0.992194,0.988406,0.992192
NearestNeighbor,NearestNeighbor,0.841233,0.856983,0.841233,0.840956,0.797502,0.840443
MLP_Baseline,MLP_Baseline,0.995006,0.995123,0.995006,0.994994,0.993607,0.99498


In [10]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BritishCarabids/6.ViLT-BritishCarabids-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BritishCarabids/6.ViLT-linear-BritishCarabids-metrics.csv", index=False)